# Who is number one?

The Massey and Colley methods of ranking presented are directly from Chapter 12 of *When Life is Linear,* by Tim Chartier.  You can access a digital copy through Westminster's Library "search" as of Fall 2024.

Additionally a (semi-)recent SIAM article by Professor Chartier on this very topic can be found at [Reflections on March Mathness](https://sinews.siam.org/Details-Page/reflections-on-march-mathness).

## The Massey method

Kenneth Massey developed this ranking scheme as an undergraduate in 1997. The method ranks teams not only on win-loss but also takes into account the point difference in each game.  Not all teams need to play the same number of games, or each other. Massey ranking has been used in the NCAA Division 1 Bowl Championship Series in the past, and these ideas still inform current ranking algorithms.

From a linear algebra point of view, Massey ranking uses least squares (regression) on a particular matrix, $M_1.$  

We define $M_1$ by setting up a matrix where each row represents a game, and each column represents a team.  For each game a "1" is placed in the column of the winner, and a "-1" in the column of the loser. If the game is a tie, we place a 1 in each column for the teams that played in that game.  For each game played (number of rows) we also construct a $p_1$ column with the point difference for the game represented by that row. Note we are subtracting the losing score from the winning score, so all point differences in this vector will be nonnegative. 

Assuming that we have a ranking of teams based on games played, we consider our ranking vector $r,$ and the matrix equation

$M_1 \mathbf{r} \quad = \quad \mathbf{p_1}.$

We assume that there exists a ranking vector, $\mathbf{r}$, giving weights to a linear combination of the columns of $M_1$ that equals the point differential vector $\mathbf{p_1}$. 

There is unlikely to be such a ranking vector, $\mathbf{r}$, which solves the system of linear equations originally described. In this case $\mathbf{p_1}$ will not be in the column space of $M_1$. 

We use projections to find the "least squares" solution to the weights given by the ranking vector $\mathbf{r}.$ The least squares solution describes the closest vector in the column space of $M_1$ to the actual point difference vector $\mathbf{p_1}$.

## Load data

We will use the [pandas](https://pandas.pydata.org/) library for its data wrangling tools while working with data obtained using the [nfl_data_py](https://github.com/cooperdff/nfl_data_py) library. We will not use `nfl_data_py` specifically in this lab but the link is provided if you want to explore those tools further.

The following code cells contains both comments and code to read in our csv file and create our initial matrix $M_1,$ and point differential vector $\mathbf{p_1}.$

Note that, $M_1$ and $\mathbf{p_1}$ are modified during the process to perform the ranking as desired.  (See comments in code cells.)

In [5]:
# load pandas and numpy
import pandas as pd
import numpy as np

In [7]:
#this will allow wider matrices to print withtout wrapping
np.set_printoptions(linewidth = 200)

In [15]:
# read in the csv file as a pandas DataFrame
nfl_2024_schedule = pd.read_csv("nfl_2024_thru_wk_03.csv")
# nfl_2024_schedule

## Data wrangling

We have more information than we need for our application.  The `nfl_2024_sched_thru_wk_03.csv` data frame has the entire pre-built regular season schedule loaded, inlcuding games that have not been played yet.  The name suggests that we have completed score data through week 3. We will filter to a subset of this data in the code cell below.

In [11]:
# there are more variables (cols) than we need
# we can select what we want as follows
wk_away_home_info = nfl_2024_schedule.loc[:, ['week', 'away_team', 'away_score', 'home_team', 'home_score']]
wk_away_home_info

,week,away_team,away_score,home_team,home_score
0,1,BAL,20.0,KC,27.0
1,1,GB,29.0,PHI,34.0
2,1,PIT,18.0,ATL,10.0
3,1,ARI,28.0,BUF,34.0
4,1,TEN,17.0,CHI,24.0
...,...,...,...,...,...
267,18,MIA,NaN,NYJ,NaN
268,18,NYG,NaN,PHI,NaN
269,18,CIN,NaN,PIT,NaN
270,18,NO,NaN,TB,NaN


Notice we now have a dataframe with 272 rows and 5 columns. We selected all 272 rows of the 2024 NFL regular season schedule.  Not all of the games have been played yet, so there are necessarily `NA`s in the dataframe.  We will further wrangle to a smaller dataframe containing the information we can code into a numpy array.

First let's try an example first with week 1 only.  There 32 teams in the NFL and all play in week 1. 

In [18]:
# create a dataframe of week 1 games only.
# note the indexing in a pd.DataFrame is similar to a np.array
week_01 = wk_away_home_info[0:16]
week_01

,week,away_team,away_score,home_team,home_score
0,1,BAL,20.0,KC,27.0
1,1,GB,29.0,PHI,34.0
2,1,PIT,18.0,ATL,10.0
3,1,ARI,28.0,BUF,34.0
4,1,TEN,17.0,CHI,24.0
5,1,NE,16.0,CIN,10.0
6,1,HOU,29.0,IND,27.0
7,1,JAX,17.0,MIA,20.0
8,1,CAR,10.0,NO,47.0
9,1,MIN,28.0,NYG,6.0


Looking at the columns and entries we see that we have `week`, `away_team`, `away_score`, `home_team`, and `home_score` variables. There is more information in the full data frame, but this is enough for us (for now.) The 32 teams in the NFL are listed below.

In [20]:
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 
         'LAC', 'LA', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

# create dictionary d from the teams "list"
d = {}
for v, k in enumerate(teams):
    d[k] = v

#print(teams)

# d is a dictionary pairing together a "value" and "key" (Team, number)
# uncover the print commands above and below to see the different output 

print("")

#print(d)

Now that we have a smaller data frame to work with let's initialize our win/loss matrix $M_1$. "Initialization" is the pre-allocation of memory ahead of filling in the data. We know how many games were played in week 1 and can use this information by attaching `.shape` in python to the data frame structure (the same as with numpy arrays.)  

In [22]:
# use the following template for creating the M1 matrix
# can modify dataframe being used in the first line

df = week_01

#############
# Initialize M1 matrix
s1 = df.shape
# shape of week 1
M1 = np.zeros([ s1[0], 33 ])
# 16 games s1[0], 32 teams, 1 point diff column

# what are s1 and M1?  How can you check?

In [26]:
# Use the following script to create M1 from whichever dataframe you specified above.
for i in range(s1[0]):
    M1[i, 32] = np.abs(df.iloc[i, 2] - df.iloc[i, 4])
    if df.iloc[i, 2] > df.iloc[i, 4]:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = 1, -1
    elif df.iloc[i, 2] < df.iloc[i, 4]:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = -1, 1
    else:
        M1[i, d[ df.iloc[i, 1]] ], M1[i, d[ df.iloc[i, 3]] ] = 1, 1

In [28]:
# check the dimensions of the matrix to make sure it is the size we think it is.
s = np.array(M1.shape)
print(s)

# create point diff vector, p, by taking last col of current M.
# We now have the point diff vector p1 described earlier.
# take all rows of M1 and the last column [:,-1]
p1 = M1[:, -1]

print(p1.shape)
print("")

#print(p1)
print("")

# Note, we still need to create our Massey matrix M by taking the last col off the original csv data.
# We now have the M1 described above with win/loss info
M1 = M1[:, 0:-1] # (s[1]-1)]
print(M1.shape)

# Pay careful attention to the rank later!

#whole code is removing the last column: point difference column only!

[16 33]
(16,)


(16, 32)


In [30]:
# vector b described for Colley ranking.
# we need this later, but create it now.
b = np.ones( s[1] - 1 ) + ( 0.5*np.sum( M1, axis=0 ) )
#print(b)

---
### Answer the following questions.

#### 1. Why do the matrix dimensions of $M_1$ change in the code above.  Explain.

> Because we stripped out the point difference column from the M1 and kept it in p1 variable.

#### 2. Describe M1.shape and what it represents in each code cell above.

> M1.shape is giving (16,32)
> 16 represents the number of rows and for week 1 there were 16 games therefore one row for each game
> 32 is the number of teams that played. So there are columns for each team in each game.. although only two of them played.

#### 3. What are the number of entries in $p_1$. Why?

> There are 31 entries in p1 because p1 contains point difference between two teams each game. And there were only 16 games in week 1. Thus, only 16 entries.

### Least squares is a projection onto the column space

Now we implement code to project $\mathbf{p_1}$ onto the column space of $M_1.$  (This is the linear algebra version of least squares regression.)

The code in the next cell forms the Massey matrix $M$ by multiplying $M_1^T$ and $M_1$ and replacing the last row of this symmetric matrix, $M\; = \; M_1^TM_1$ with ones. Since we are mulitplying our matrix $M_1$ on the left by its transpose we need to do the same on the right hand side of the original matrix equation 

$M_1\mathbf{r} \; = \; \mathbf{p_1}.$

So we now have

$M^T_1 M_1 \mathbf{r} \; = \; M_1^T \mathbf{p_1},$

and by replacing the last row as described, we get

$M \mathbf{r} \; = \; \mathbf{p}.$

If $M$ is invertible (it should be by our construction) we can solve for our ranking vector $\mathbf{r}.$

In [42]:
# Code here forms the symmetric matrix M1.T@M1.  This is how we form the normal equation to solve a least squares problem. 
# We are projecting p1 onto the column space of M1 to find the best fitting ranking vector to our game data.

# (32 x 16)(16 x 32) = (32 x 32)
sm1 = np.array((M1.T@M1).shape)
print(sm1)
print("")

#number of pivots is matrix_rank: number of independent columns
print(np.linalg.matrix_rank(M1.T@M1))
print("")

[32 32]

16



In [52]:
# The Massey implementation of least squares is slightly different than a usual least squares. 
# We multiply both sides of the matrix equation by M1 transpose, M1.T, giving the new vector p=M1.T@p1 on the left, 
# and the symmetric matrix M=M1.T@M1 on the right (usual least squares.)

# The difference is that we now replace the last entry of p with a zero, 
# and the last row of M with a row of ones.  
# This adds the condition that the sum of the ranks will be zero.

# create vector p as described above.
p = M1.T@p1
p[-1,] = 0

# we are getting here because:
# M1 . r = p1
# M1t . M1 . r = M1t . p1
# M . r = p

# Next step to get M
M = M1.T@M1

# C is the Colley matrix.  We will use this for another method shortly.
C = M+2*np.identity(32);

# Final step for creating the Massey matrix M
M[-1,:] = np.ones(sm1[1])
print("")

# check rank of M
print(np.linalg.matrix_rank(M))


17


In [54]:
# Our Ax=b is M@r = p at this point. We use r = np.linalg.lstsq(M,p, rcond=None)[0]
# to implement a safer version of x = linalg.solve(A,b) to calculate x for solving linear systems.  
# Do not use any inverse commands!
#
# (MATLAB warning: We use x = A\b to calculate r for solving linear systems.  Do not use inv().)
#
# r is the ranking vector that 'best' fits the data.  This is least squares regression.

r = np.linalg.lstsq(M, p, rcond=None)[0]

print(r.shape)

print("")

# This is your ranking vector r
print(r)

print("")

# check to see that the sum of ranks is zero.
print(sum(r))

(32,)

[ -3.   -4.   -3.5   3.  -18.5   3.5  -3.   -8.    8.   -3.    3.   -2.5   1.   -1.   -1.5   3.5   6.   -3.   -6.    1.5  11.    3.   18.5 -11.   -6.5   2.5   4.    3.    6.5   8.5  -3.5  -8.5]

5.329070518200751e-15


### NFL Teams


Teams listed by alphabetical order indicating their column in our matrices. Note that the Los Angeles Rams are listed as LA on our list versus LAR on the NFL website.  This originally caused some issues in our original data.


<table>
 <tr>
     <td>1. ARI</td>
     <td>2. ATL</td>
     <td>3. BAL</td>
     <td>4. BUF</td>
     <td>5. CAR</td>
     <td>6. CHI</td>
     <td>7. CIN</td>
     <td>8. CLE</td>
     <td>9. DAL</td>
     <td>10. DEN</td>
     <td>11. DET</td>
     <td>12. GB</td>
     <td>13. HOU</td>
     <td>14. IND</td>
     <td>15. JAX</td>
     <td>16. KC</td>
 </tr>
 <tr>
     <td>17. LAC</td>
     <td>18. LA</td>
     <td>19. LV</td>
     <td>20. MIA</td>
     <td>21. MIN</td>
     <td>22. NE</td>
     <td>23. NO</td>
     <td>24. NYG</td>
     <td>25. NYJ</td>   
     <td>26. PHI</td>
     <td>27. PIT</td>
     <td>28. SEA</td>
     <td>29. SF</td>
     <td>30. TB</td>
     <td>31. TEN</td>
     <td>32. WAS</td>
 </tr>
</table>



In [46]:
df2 = pd.DataFrame(
    {
        "Team" : teams,
        "Massey_ranking" : r
    }
)

In [48]:
df2_sorted = df2.sort_values(by = "Massey_ranking", ascending = False, ignore_index = True)
df2_sorted

,Team,Massey_ranking
0,NO,18.5
1,MIN,11.0
2,TB,8.5
3,DAL,8.0
4,SF,6.5
5,LAC,6.0
6,PIT,4.0
7,CHI,3.5
8,KC,3.5
9,DET,3.0


## The Colley method

The Colley method is another ranking scheme that has been used for BCS college football rankings in the past.  It does not take into account point differentials, only wins and losses. 

The linear system

$C \mathbf{r} \; = \; \mathbf{b}$

descibes the method, where $C$ is the Colley matrix, $\mathbf{r}$ is the ranking vector, and $\mathbf{b}$ contains team win/loss information.  Each entry in the vector $\mathbf{b}$ is given by

$\displaystyle{b_i \; = \; 1 +\frac{w_i-l_i}{2}}$

where $w_i$ and $l_i$ are the wins and losses for team $i.$

The Colley matrix $C$ is straightforward to compute once you have the Massey matrix $M.$ (See the description above for $M \; = \; M_1^TM_1$ with the substitution of ones in the last row.)  

The Colley matrix $C$ is given by

$C \; = \; M+2I.$

Note that the ratings are possibly different between the two schemes.  The average of the Colley ratings is 1/2, and the sum of the Massey ratings is 0.

In [56]:
# Colley ranking rc is found here.
rc = np.linalg.solve(C,b)
print(rc)
print("")

# ratings average to 1/2
print(sum(rc)/32)

[0.375 0.375 0.375 0.625 0.375 0.625 0.375 0.375 0.625 0.375 0.625 0.375 0.625 0.375 0.375 0.625 0.625 0.375 0.375 0.625 0.625 0.625 0.625 0.375 0.375 0.625 0.625 0.625 0.625 0.625 0.375 0.375]

0.5


In [58]:
# check rank of C
print(np.linalg.matrix_rank(C))

# Using inverses (just to check, in practice don't do this!)

rc1 = np.linalg.inv(C.T@C)@C.T@b
rc1

32


array([0.375, 0.375, 0.375, 0.625, 0.375, 0.625, 0.375, 0.375, 0.625, 0.375, 0.625, 0.375, 0.625, 0.375, 0.375, 0.625, 0.625, 0.375, 0.375, 0.625, 0.625, 0.625, 0.625, 0.375, 0.375, 0.625, 0.625,
       0.625, 0.625, 0.625, 0.375, 0.375])

In [60]:
df3 = pd.DataFrame(
    {
        "Team" : teams,
        "Colley_ranking" : rc1
    }
)

In [62]:
df3_sorted = df3.sort_values(by = "Colley_ranking", ascending = False, ignore_index = True)
df3_sorted

,Team,Colley_ranking
0,LAC,0.625
1,MIA,0.625
2,NE,0.625
3,KC,0.625
4,NO,0.625
5,HOU,0.625
6,PHI,0.625
7,DET,0.625
8,PIT,0.625
9,DAL,0.625


---
## Your turn

Construct the Massey and Colley rankings for 

1. the week 3 game data (need through row 48, (why?).)  We will do this together. 
2. using this information make some predictions for the week 4 set of games!
3. Predict tonight's Thursday night game and the Monday Night game using the rankings we produce.
4. Finally, answer questions below for rankings produced.
5. We will revisit next week!


For all steps above, compare the rankings you produce between the Massey and Colley methods. Find a ranking that is the same between the two methods and one that is different. (Using all three weeks of games played so far in the 2024 nfl regular season.) 

Describe in (your) words why this is likely to be happening, both from a real-world and linear algebra point of view. This description does not have to be overly technical.  Descibe in your own words. Discuss with your classmates.  

(See Ch 12 of Tim Chartier's "When Life is Linear.")

You will need to add your own code and markdown cells below. You may copy/paste from above. You may work alone or with one other classmate. (No groups of three.)

> Week 3 data needs upto row 48 because first 16 is upto week 1, the second 16 is upto week 2, and the third 16 is upto week 3. Thus three sets of 16 games is 48 games altogether.

>For Thursday Game (DAL/NYG):
>> Massey Method: DAL Wins NYG Loses
>>
>> Colley Method: NYG Wins DAL Loses
>> 
>For Monday Game (TEN/MIA):
>> Massey Method: TEN Wins MIA Loses
>> 
>> Colley Method: MIA Wins TEN Loses
>> 
>For Monday Game (SEA/DET):
>> Massey Method: DET Wins SEA Loses
>> 
>> Colley Method: SEA Wins DET Loses

#### Same Ranking
> MIN (2nd For Both Methods)
#### Different Ranking
> NO (1st in Massey, 2nd in Colley)

#### Reason
> We are only considering the point difference while calculating the predictions. However, a strong team may also win by point difference of 1 and in our algorithm, we are not considering how often they win but by how much they are winning by. Also, the linear models for predictions do not consider all the unpredictable factors like a player's injuries, weather conditions, etc. Likewise, if a player is newly added to the team, they may not have a strong history to predict the future.